In [26]:
import pandas as pd
import requests

In [145]:
# переменные

API_URL = 'https://data-charts-api.hexlet.app/'
START_DATE = '2023-03-01'
END_DATE = '2023-09-01'
VISITS_FOR_THE_PERIOD = f'{API_URL}visits?begin={START}&end={END}'
REGISTRATIONS_FOR_THE_PERIOD = f'{API_URL}registrations?begin={START}&end={END}'
VISITS_AGENTS_TO_AVOID = ['bot']

In [78]:
# запрашиваем данные, переводим их в дф

request_visits = requests.get(VISITS_FOR_THE_PERIOD)
request_regs = requests.get(REGISTRATIONS_FOR_THE_PERIOD)
visits_df = pd.DataFrame(request_visits.json())
regs_df = pd.DataFrame(request_regs.json())

In [80]:
visits_df.head(2)

,datetime,platform,user_agent,visit_id
0,"Wed, 01 Mar 2023 10:36:22 GMT",web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1de9ea66-70d3-4a1f-8735-df5ef7697fb9
1,"Wed, 01 Mar 2023 06:25:00 GMT",web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,f149f542-e935-4870-9734-6b4501eaf614


In [41]:
regs_df.head(2)

,datetime,email,platform,registration_type,user_id
0,"Wed, 01 Mar 2023 07:40:13 GMT",ebyrd@example.org,web,google,2e0f6bb8-b029-4f45-a786-2b53990d37f1
1,"Wed, 01 Mar 2023 13:14:00 GMT",knightgerald@example.org,web,email,f007f97c-9d8b-48b5-af08-119bb8f6d9b6


Фильтруем, группируем данные, мерджим общий дф, считаем конвертацию посещение-регистрация

In [229]:
# создаем копии оригинальный фреймов для последующих преобразований
vis, reg =  visits_df.copy(), regs_df.copy()

# преобразование дат к типу datetime, убираем точное время
vis['date_group'] = pd.to_datetime(vis['datetime']).dt.date
reg['date_group'] = pd.to_datetime(reg['datetime']).dt.date

# фильтр посетителей по User-Agent
vis = vis[~vis['user_agent'].str.contains('|'.join(VISITS_AGENTS_TO_AVOID), case=False)] 

# оставляем последние посещения уникальных юзеров сортируем по id и дате
vis = vis.sort_values(by=['visit_id', 'date_group']).drop_duplicates(subset=['visit_id'], keep='last')

#группируем дф'ы
gr_vis = vis.groupby(['date_group', 'platform']).size().reset_index(name='visits')
gr_reg = reg.groupby(['date_group', 'platform']).size().reset_index(name='registrations')

# мерджим visits и registrations
df = pd.merge(gr_vis, gr_reg,
                      on=['date_group', 'platform'],
                      how='outer')

df['conversation'] = (df['registrations'] / df_vis_reg['visits']  * 100).round(2)

#df

df.to_json('./conversion.json')

In [227]:
df

,date_group,platform,visits,registrations,conversation
0,2023-03-01,android,75,61,81.33
1,2023-03-01,ios,22,18,81.82
2,2023-03-01,web,279,8,2.87
3,2023-03-02,android,67,59,88.06
4,2023-03-02,ios,31,24,77.42
...,...,...,...,...,...
547,2023-08-30,ios,66,40,60.61
548,2023-08-30,web,1227,34,2.77
549,2023-08-31,android,57,42,73.68
550,2023-08-31,ios,50,36,72.00


In [120]:
vis.head(6)


,datetime,platform,user_agent,visit_id
0,2023-03-01 10:36:22,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1de9ea66-70d3-4a1f-8735-df5ef7697fb9
1,2023-03-01 06:25:00,web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,f149f542-e935-4870-9734-6b4501eaf614
2,2023-03-01 10:26:13,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1
3,2023-03-01 12:33:06,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,19322fed-157c-49c6-b16e-2d5cabeb9592
4,2023-03-01 01:38:35,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,04762a22-3c9f-40c9-9ac9-6628c4381836
5,2023-03-01 02:11:55,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,040e1e30-c9ed-4248-bc97-99a707e36d60


In [97]:
reg.head(2)

,datetime,email,platform,registration_type,user_id
0,2023-03-01 07:40:13,ebyrd@example.org,web,google,2e0f6bb8-b029-4f45-a786-2b53990d37f1
1,2023-03-01 13:14:00,knightgerald@example.org,web,email,f007f97c-9d8b-48b5-af08-119bb8f6d9b6
